## Getting Started: Analyzing Yelp Data

In [ ]:
%%read s3  --as yelp_biz --cache True
path: ps-samples/yelp/businesses/json
format: json

In [ ]:
%%sql
SELECT name,city,stars,review_count
FROM yelp_biz
WHERE stars > 3.5
SORT BY review_count DESC 
LIMIT 30

In [ ]:
%%sql -a nevada_yelp_biz
SELECT state, city, sum(review_count) as totalreviews
FROM yelp_biz
where attributes.`Good For`.dinner = true 
GROUP By state, city
ORDER By totalreviews desc

In [ ]:
%%plot bar --data nevada_yelp_biz  --limit 20 
x : city
y : totalreviews
xTitle : city
yTitle : Total Reviews
title : Yelp Reviews Count By City
layout:
    height : 500

### We can do all of the above code as a single line python code too

In [ ]:
read(kind="s3", path="ps-samples/yelp/businesses/json", format="json" ,cache=True)\
.groupBy("city")\
.agg(sum("review_count").alias("totalreviews"))\
.sort(desc("totalreviews"))\
.plot(kind="bar", barmode="stack", limit=20, x="city", y=["totalreviews"], 
      xTitle = "City", yTitle="Total Reviews", title="Yelp Reviews Count By City")

In [ ]:
table("yelp_biz").groupBy("stars").agg({"*": "count"}).sort("stars").plot(kind="scatter",x="stars",y="count(1)")

### Working with multiple data sources together

In [ ]:
%%read s3 --as yelp_user --cache True
path : ps-samples/yelp/users/json 
format: json

In [ ]:
%%read s3 --as yelp_review 
path: ps-samples/yelp/reviews/json
format: json

In [ ]:
%%sql 
SELECT yelp_biz.business_id, yelp_biz.city, 
       yelp_review.text, yelp_review.stars, yelp_review.date 
FROM yelp_biz 
JOIN yelp_review ON (yelp_biz.business_id = yelp_review.business_id) 
LIMIT 12

In [ ]:
%%sql 
SELECT yelp_biz.name,yelp_biz.city,yelp_biz.review_count,
       yelp_review.text, yelp_review.stars, yelp_review.date, yelp_user.average_stars, 
       yelp_user.review_count, yelp_user.fans
FROM yelp_biz 
JOIN yelp_review ON (yelp_biz.business_id = yelp_review.business_id) 
JOIN yelp_user ON (yelp_user.user_id = yelp_review.user_id)  
LIMIT 3

In [ ]:
registerFunction("hasFourParts", lambda x: len(x.split(" ")) >= 4,  BooleanType())

In [ ]:
%%sql
SELECT name,stars,city,review_count 
FROM yelp_biz 
WHERE hasFourParts(name)
SORT BY review_count 
DESC LIMIT 10

## Machine learning

In [ ]:
bin = Binarizer(inputCol = "stars", outputCol = "label", threshold = 3.5)
tok = Tokenizer(inputCol = "text", outputCol = "words")
hashTF = HashingTF(inputCol = tok.getOutputCol(), numFeatures = 10000, outputCol = "features")
lr = LogisticRegression(maxIter = 10, regParam = 0.0001, elasticNetParam = 1.0)
pipeline = Pipeline(stages = [bin, tok, hashTF, lr])

In [ ]:
preppedReviews = table("yelp_review").limit(100).map(lambda row: Row(text=row.text, stars=float(row.stars))).toDF()

In [ ]:
model = pipeline.fit(preppedReviews)

In [ ]:
model.transform(preppedReviews).select("label","prediction","probability","stars","text").plot(limit=20)

## Custom ETL

In [ ]:
words = table("yelp_review").limit(1000).select("text").flatMap(lambda row: row.text.split(" ")).map(lambda w: Row(word=w, cnt=1)).toDF()

In [ ]:
words.groupBy("word").sum().orderBy(desc("SUM(cnt)")).plot(limit=10)

## Other Magics

In [ ]:
%%tables

In [ ]:
%%schema -d yelp_biz

In [ ]:
%%read redshift
path

In [ ]:
%%s3